In [ ]:
# setup and imports
import time
import numpy as np
from numpy import fft
from numpy.fft import fftshift, fftfreq
import matplotlib.mlab as mlab

In [ ]:
# overlay setup
from rfsoc_mts import mtsOverlay
ol = mtsOverlay('./fpga_bit_and_hwh/mts_experiment13c_adc_accumulate.bit')
ol.init_tile_sync()
ol.verify_clock_tree()
ol.sync_tiles()

In [ ]:
# DAC Play
ol.gpio_dac_play_numSampleVectors    = ol.hier_dac_play.axi_gpio_numSampleVectors.channel1
ol.gpio_dac_play1_numSampleVectors   = ol.hier_dac_play1.axi_gpio_numSampleVectors.channel1

# Add DAC B
ol.dac_player1 = ol.memdict_to_view("hier_dac_play1/axi_bram_ctrl_0")
ol.dac_capture1 = ol.memdict_to_view("hier_dac_cap1/axi_bram_ctrl_0")

ol.gpio_dac_play_numSampleVectors    = ol.hier_dac_play.axi_gpio_numSampleVectors.channel1
ol.gpio_dac_play1_numSampleVectors   = ol.hier_dac_play1.axi_gpio_numSampleVectors.channel1



[hex(ol.gpio_dac_play_numSampleVectors.read()), 
 hex(ol.gpio_dac_play1_numSampleVectors.read())]  # By default, set to something like 0xFFF for bitstream

In [ ]:
# There is one DAC accumulate, but 2 ADC accumulates (even though there are 2 DAC plays)

# DAC inputs to Verilog block:
ol.gpio_dac_accumulate_samples_in_window   = ol.hier_dac_accumulate.axi_gpio_samples_in_window.channel1
ol.gpio_dac_accumulate_num_accumulations   = ol.hier_dac_accumulate.axi_gpio_num_accumulations.channel1
ol.gpio_dac_accumulate_reading_dump        = ol.hier_dac_accumulate.axi_gpio_reading_dump.channel1[0]  # single bit
# DAC outputs of Verilog block:
ol.gpio_dac_accumulate_dump_ready          = ol.hier_dac_accumulate.axi_gpio_dump_ready.channel1[0]  # single bit
ol.gpio_dac_accumulate_sequence_number     = ol.hier_dac_accumulate.axi_gpio_sequence_number.channel1
ol.gpio_dac_accumulate_overflow_count      = ol.hier_dac_accumulate.axi_gpio_overflow_count.channel1
ol.gpio_dac_accumulate_dropped_count       = ol.hier_dac_accumulate.axi_gpio_dropped_count.channel1
# DAC memory
ol.dac_accumulate = ol.memdict_to_view("hier_dac_accumulate/axi_bram_ctrl_0", dtype='int32')

# ADC0 inputs to Verilog block:
ol.gpio_adc0_accumulate_samples_in_window   = ol.hier_adc0_accumulate.axi_gpio_samples_in_window.channel1
ol.gpio_adc0_accumulate_num_accumulations   = ol.hier_adc0_accumulate.axi_gpio_num_accumulations.channel1
ol.gpio_adc0_accumulate_reading_dump        = ol.hier_adc0_accumulate.axi_gpio_reading_dump.channel1[0]  # single bit
# ADC0 outputs of Verilog block:
ol.gpio_adc0_accumulate_dump_ready          = ol.hier_adc0_accumulate.axi_gpio_dump_ready.channel1[0]  # single bit
ol.gpio_adc0_accumulate_sequence_number     = ol.hier_adc0_accumulate.axi_gpio_sequence_number.channel1
ol.gpio_adc0_accumulate_overflow_count      = ol.hier_adc0_accumulate.axi_gpio_overflow_count.channel1
ol.gpio_adc0_accumulate_dropped_count       = ol.hier_adc0_accumulate.axi_gpio_dropped_count.channel1
# ADC0 memory
ol.adc0_accumulate = ol.memdict_to_view("hier_adc0_accumulate/axi_bram_ctrl_0", dtype='int32')


# ADC1 inputs to Verilog block:
ol.gpio_adc1_accumulate_samples_in_window   = ol.hier_adc1_accumulate.axi_gpio_samples_in_window.channel1
ol.gpio_adc1_accumulate_num_accumulations   = ol.hier_adc1_accumulate.axi_gpio_num_accumulations.channel1
ol.gpio_adc1_accumulate_reading_dump        = ol.hier_adc1_accumulate.axi_gpio_reading_dump.channel1[0]  # single bit
# ADC1 outputs of Verilog block:
ol.gpio_adc1_accumulate_dump_ready          = ol.hier_adc1_accumulate.axi_gpio_dump_ready.channel1[0]  # single bit
ol.gpio_adc1_accumulate_sequence_number     = ol.hier_adc1_accumulate.axi_gpio_sequence_number.channel1
ol.gpio_adc1_accumulate_overflow_count      = ol.hier_adc1_accumulate.axi_gpio_overflow_count.channel1
ol.gpio_adc1_accumulate_dropped_count       = ol.hier_adc1_accumulate.axi_gpio_dropped_count.channel1
# ADC1 memory
ol.adc1_accumulate = ol.memdict_to_view("hier_adc1_accumulate/axi_bram_ctrl_0", dtype='int32')

In [ ]:
# bokeh setup
import bokeh
import bokeh.plotting
from bokeh.io import push_notebook, show, output_notebook
# from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
# import bokeh.palettes
output_notebook()

In [ ]:
# ZC functions

def zc_signal(sample_rate, nSamples, u=17, q=0, f_start=200e6, f_stop=800e6):
    '''
    returns a real signal, where a ZC sequence is created and "upconverted" to
    have a frequency spectrum between f_start and f_stop.
    
    This is all done in the FFT domain, by copying and pasting the 
    FFT of the ZC sequence into the appropriate bins of the target signal,
    which is nSamples at sample_rate.
    
    Often there isn't a spectral bin that exactly corresponds to f_start and f_stop,
    so we find the cloest ones.
    
    This all means that the actual integer length of the ZC sequence (NZC) depends 
    on all of these parameters, including the rounding to the nearest spectral bin.
    
    You do NOT want a window here for the repeated, circular correlation case.
    '''
    txfft_shifted_freq = fftshift(fftfreq(nSamples, 1/sample_rate))
    
    # Find the bins most closely corresponding to f_start and f_stop (inclusive)
    start_bin = np.argmin(np.abs(txfft_shifted_freq - f_start))
    stop_bin = np.argmin(np.abs(txfft_shifted_freq - f_stop))
    start_delta = txfft_shifted_freq[start_bin]-f_start
    stop_delta = txfft_shifted_freq[stop_bin]-f_stop
    print('f_start:', f_start, ' actual:', txfft_shifted_freq[start_bin], ' delta:', start_delta, 'or', start_delta/f_start*100, '%')
    print('f_stop:', f_stop, ' actual:', txfft_shifted_freq[stop_bin], ' delta:', stop_delta, 'or', stop_delta/f_stop*100, '%')
    
    neg_start_bin = np.where(txfft_shifted_freq == -txfft_shifted_freq[start_bin])[0][0]
    neg_stop_bin  = np.where(txfft_shifted_freq == -txfft_shifted_freq[stop_bin])[0][0]
    
    # The ZC sequence fills exactly these bins
    NZC = stop_bin - start_bin + 1
    print('NZC:', NZC, '  with small_factors:', small_factors(NZC)[1:])
    zu = zc_sequence(NZC, u, q)  # complex sequence
    
    # check that the spectrum is flat and with magnitude equak to NZC
    zufft = np.fft.fft(zu)
    assert( len(zufft) == NZC )
    # the way this FFT is normalized, it's mag^2 is NZC
    zzstar = zufft * zufft.conjugate()
    assert( np.allclose(zzstar, NZC) )
    
    # Check that the auto correlation is a Kronecker delta.
    auto_corr = np.fft.ifft(zzstar)
    kronecker = np.zeros_like(auto_corr)
    kronecker[0] = 1.0
    assert( np.allclose(auto_corr, kronecker*NZC, rtol=1e-05, atol=1e-05) )  # allowed looser rounding for RFSoC ARM
    
    # We want a version of the ZC sequence's FFT with the zero bin in the center
    zufft_shifted = fftshift(zufft)
    
    # start new longer complex fft array for one sequence worth of TX samples as containing all zeros.
    txfft_shifted = np.zeros(nSamples, dtype=zufft.dtype)
    # now "paste" in the sequence FFT at the appropriate spot in the longer tx FFT.
    txfft_shifted[start_bin:stop_bin+1] = zufft_shifted
    # now "paste" in the sequence FFT's reverse conjugate at the appropriate spot in the longer tx FFT.
    txfft_shifted[neg_stop_bin:neg_start_bin+1] += zufft_shifted[::-1].conjugate()

    txfft = fftshift(txfft_shifted)
    # optionally zero out DC bin here. Everything is AC coupled anyway and there's no use wasting power on it.
    txfft[0] = 0.0

    # Inverse FFT and check that the result is real
    tx_complex = np.fft.ifft(txfft)
    assert(np.allclose(tx_complex.imag, 0))  # Check that the IFFT is real
    tx = tx_complex.real # the real transmitted signal
    # TODO: maybe repeat this Nseq times before loading into TX DAC buffer. Use np.tile()
    return tx

def small_factors(n):
    mods = np.array(list(n % i for i in range(1, int(np.sqrt(n)) + 1)))
    return np.where(mods==0)[0]+1

def zc_sequence(NZC, u=17, q=0):
    '''
    returns a complex Zadoff-Chu sequence
    according to https://en.wikipedia.org/wiki/Zadoff%E2%80%93Chu_sequence
    This is the "ideal ZC sequence" or "Wikipedia ZC sequence" or "textbook ZC sequence"

    u is the interesting prime parameter
    q is a kind of shift. 
    q=0 is just a Chu sequence. Non-zero q is just cyclically-shifted and constant-phased
    '''
    # Boring parameters
    cf = NZC%2
    #assert(gcd(NZC,u)==1)  # If this fails, the spectrum won't be flat! It'll have periodic structure
    n = np.arange(NZC)  # index of sequence element. Others would call this i
    zu = np.exp(-1j*np.pi*u*n*(n+cf+2*q)/NZC)
    # Check that it's periodic even if NZC is even (even though Wikipedia clames otherwise)
    zu_next_element_off_the_end = np.exp(-1j*np.pi*u*NZC*(NZC+cf+2*q)/NZC)
    assert( np.isclose(zu[0], zu_next_element_off_the_end) )
    return zu

In [ ]:
nSamples = len(ol.dac_player)
print('nSamples:', nSamples)
sample_rate = 4.0E9
DAC_SR = 4.0E9  # Sample rate of DACs and ADCs is 4.0 GHz
ADC_SR = 4.0E9
assert(DAC_SR==sample_rate)
assert(ADC_SR==sample_rate)
#Fc = 250.0E6 # Set center frequency of waveform to 250.0 MHz; also start of chirp
Fc = 10.0E6 # Set center frequency of waveform to 10.0 MHz; also start of chirp
Fe = 500.0E6 # maximum frequency of chirp at end of record
DAC_Amplitude = 16383.0  # 2**14 14bit DAC +16383/-16384
#DAC_Amplitude = 2**15  # 16bit signed DAC
X_axis = (1/DAC_SR) * np.arange(0,nSamples)

f_start, f_stop = 0, 1800e6  # test
u1, u2 = 1, 1
DAC_zc1 = zc_signal(sample_rate=sample_rate, nSamples=nSamples, u=u1, q=0, f_start=f_start, f_stop=f_stop)
DAC_zc1 = DAC_zc1 * DAC_Amplitude / np.max(np.abs(DAC_zc1))
DAC_zc2 = zc_signal(sample_rate=sample_rate, nSamples=nSamples, u=u2, q=0, f_start=f_start, f_stop=f_stop)
DAC_zc2 = DAC_zc2 * DAC_Amplitude / np.max(np.abs(DAC_zc2))


In [ ]:
# set up dummy data to be updated later
spectrum_source_0 = ColumnDataSource(
    data={
        "freqs_MHz":  [100, 200, 300],  # x
        "powers_dB":  [10, 20, 30],  # y
    }
)

spectrum_source_1 = ColumnDataSource(
    data={
        "freqs_MHz":  [100, 200, 300],  # x
        "powers_dB":  [10, 20, 30],  # y
    }
)

In [ ]:
# spectrum figure setup
p = figure(
        height=400,
        title='spectrum',
        x_axis_label='freq [MHz]',
        y_axis_label='power [dB]',
        y_range=(-70,0),
        tools=("pan", "zoom_in", "zoom_out", "box_zoom", "wheel_zoom", "save", "reset", "help", "hover"),
        sizing_mode="stretch_width",
        tooltips="@freqs_MHz, @powers_dB"  # define a tooltip using data from the x and y columns
    )

p.line(
    x="freqs_MHz",  # use the sequence in the "x_values" column
    y="powers_dB",  # use the sequence in the "y_values" column
    source=spectrum_source_0,  # use the ColumnDataSource as the data source
)

p.line(
    x="freqs_MHz",  # use the sequence in the "x_values" column
    y="powers_dB",  # use the sequence in the "y_values" column
    source=spectrum_source_1,  # use the ColumnDataSource as the data source
    line_color="coral"

)
# show the results, which will get updated in real time later
spectrum_handle = show(p, notebook_handle=True)

In [ ]:
# for now, assume lengths of accumulate buffers are identical.
assert( len(ol.dac_accumulate) % 16 == 0)  # they must also be multiples of 16 samples.
assert( len(ol.dac_accumulate) == len(ol.adc0_accumulate) )
assert( len(ol.dac_accumulate) == len(ol.adc0_accumulate) )

# set samples_in_window to completely fill the buffers
ol.gpio_dac_accumulate_samples_in_window.write(len(ol.dac_accumulate), 0xFFFFFFFF)
ol.gpio_adc0_accumulate_samples_in_window.write(len(ol.adc0_accumulate), 0xFFFFFFFF)
ol.gpio_adc1_accumulate_samples_in_window.write(len(ol.adc1_accumulate), 0xFFFFFFFF)

# fixed-pattern inputs
#ol.dac_player[:] = np.array([0]*len(ol.dac_player))  # zeros
#ol.dac_player[:] = np.array([1]*len(ol.dac_player))  # ones
#ol.dac_player[:] = np.array([-1]*len(ol.dac_player))  # -1s
#ol.dac_player[:] = np.arange(len(ol.dac_player))  # 0,1,2,3...  this rolls over to negative values for 256k memory
#ol.dac_player[:] = (np.arange(len(ol.dac_player))%2)+MAX_INT16  # MAX,MIN,MAX,MIN rolls over right away, even with small memory
#ol.dac_player[:] = MAX_INT16*np.cos(2*np.pi*np.arange(len(ol.dac_player))/2**10)
#ol.dac_player[:] = DAC_chirp
#ol.dac_player[:] = DAC_zc1
#ol.dac_player[:] = np.zeros_like(DAC_zc1)

# copy to the other DAC  (DAC B)
#ol.dac_player1[:] = ol.dac_player[:]
#ol.dac_player1[:] = (2**15-1)*np.cos(2*np.pi*np.arange(len(ol.dac_player))/100.0) # override with faster cos
ol.dac_player1[:] = DAC_zc2
# Choose num_accumulations. Too many can cause overflow and underflow. Test for this.
# num_accumulations = 1000  # fast, definitely no overflow # 3000 is too fast (almost 60 Hz) #4000 is 30 Hz
num_accumulations = 10000
# 3000 is almost 60 Hz
# 4000 is 30 Hz
# 10000 is 12.2 HZ

ol.gpio_dac_accumulate_num_accumulations.write(num_accumulations, 0xFFFFFFFF)
ol.gpio_adc0_accumulate_num_accumulations.write(num_accumulations, 0xFFFFFFFF)
ol.gpio_adc1_accumulate_num_accumulations.write(num_accumulations, 0xFFFFFFFF)

# Empty lists to be filled in loop
accumulations_dac = []
accumulations_adc0 = []
accumulations_adc1 = []
sequence_number = []
overflow_count = []
dropped_count = []

# collect the data
ol.trigger_capture()  # Need to call this to output anything at all.
ol.trig_cap.on()

ol.gpio_dac_accumulate_reading_dump.off()
ol.gpio_adc0_accumulate_reading_dump.off()
ol.gpio_adc1_accumulate_reading_dump.off()


while True:  # until you hit stop....
    # Wait for a rising edge on dump_ready, then copy out the BRAM as fast as possible
    print("waiting....")
    while ol.gpio_dac_accumulate_dump_ready.read() != 1:
        #print("waiting for dump_ready")
        time.sleep(0.0001)
    print("done waiting")
        #time.sleep(dump_time*1.5) # BAD: causes missed sequences (but not drops) on purpouse to test.
    ol.gpio_dac_accumulate_reading_dump.on()  # Tell the hardware that we are reading now
    # next, copy the accumulated data from hardware BRAM into ARM's RAM.
    # if you don't copy the array, old pointers will stick around and using these will kill the kernel!
    accumulations_dac = ol.dac_accumulate.copy()
    #time.sleep(dump_time*1.1)  # BAD: waiting anywhere near dump_time or longer will drop samples
    sequence_number.append( ol.gpio_dac_accumulate_sequence_number.read() )
    overflow_count.append(  ol.gpio_dac_accumulate_overflow_count.read()  )
    dropped_count.append(   ol.gpio_dac_accumulate_dropped_count.read()   )
    ol.gpio_dac_accumulate_reading_dump.off() # Tell the hardware that we are done reading. This used to be done before reading these three status registers, but that's probably wrong.

    # Wait for a rising edge on dump_ready, then copy out the BRAM as fast as possible
    while ol.gpio_adc0_accumulate_dump_ready.read() != 1:
        time.sleep(0.0001)
        #time.sleep(dump_time*1.5) # BAD: causes missed sequences (but not drops) on purpouse to test.
    ol.gpio_adc0_accumulate_reading_dump.on()  # Tell the hardware that we are reading now
    # next, copy the accumulated data from hardware BRAM into ARM's RAM.
    # if you don't copy the array, old pointers will stick around and using these will kill the kernel!
    accumulations_adc0 = ol.adc0_accumulate.copy()
    #time.sleep(dump_time*1.1)  # BAD: waiting anywhere near dump_time or longer will drop samples
    sequence_number.append( ol.gpio_adc0_accumulate_sequence_number.read() )
    overflow_count.append(  ol.gpio_adc0_accumulate_overflow_count.read()  )
    dropped_count.append(   ol.gpio_adc0_accumulate_dropped_count.read()   )
    ol.gpio_adc0_accumulate_reading_dump.off() # Tell the hardware that we are done reading. This used to be done before reading these three status registers, but that's probably wrong.
    
    # Wait for a rising edge on dump_ready, then copy out the BRAM as fast as possible
    while ol.gpio_adc1_accumulate_dump_ready.read() != 1:
        time.sleep(0.0001)
    ol.gpio_adc1_accumulate_reading_dump.on()  # Tell the hardware that we are reading now
    # next, copy the accumulated data from hardware BRAM into ARM's RAM.
    # if you don't copy the array, old pointers will stick around and using these will kill the kernel!
    accumulations_adc1 = ol.adc1_accumulate.copy()
    sequence_number.append( ol.gpio_adc1_accumulate_sequence_number.read() )
    overflow_count.append(  ol.gpio_adc1_accumulate_overflow_count.read()  )
    dropped_count.append(   ol.gpio_adc1_accumulate_dropped_count.read()   )
    ol.gpio_adc1_accumulate_reading_dump.off() # Tell the hardware that we are done reading. This used to be done before reading these three status registers, but that's probably wrong.
    
    #cor_0 = periodic_corr(ol.dac_player1, accumulations_adc0/num_accumulations)
    #cor_1 = periodic_corr(ol.dac_player1, accumulations_adc1/num_accumulations)

    
    #print(sequence_number[-3:])
    #print(overflow_count[-3:])
    #print(dropped_count[-3:])

    
    (Pxx_accumulations_adc0, freqs_0) = mlab.psd(accumulations_adc0/num_accumulations, NFFT=2**8, Fs=4e9, window=mlab.window_hanning)
    (Pxx_accumulations_adc1, freqs_1) = mlab.psd(accumulations_adc1/num_accumulations, NFFT=2**8, Fs=4e9, window=mlab.window_hanning)
    
    # from matplot: For plotting, the power is plotted as 10*log10(Pxx) for decibels, though Pxx itself is returned.
    
    print(freqs_0[5:8], 10*np.log10(Pxx_accumulations_adc0)[5:8])
    print(sequence_number)
    spectrum_source_0.data = {
        "freqs_MHz" : freqs_0/1e6, # x
        "powers_dB" : 10*np.log10(Pxx_accumulations_adc0), # y
    }
    
    spectrum_source_1.data = {
        "freqs_MHz" : freqs_1/1e6, # x
        "powers_dB" : 10*np.log10(Pxx_accumulations_adc1), # y
    }

    # push and update plot
    #print("pushing to notebook")
    push_notebook(handle=spectrum_handle)
    
    # At 4 GSPS, what is the dump rate per second?
    dump_rate = 4e9 / ol.gpio_dac_accumulate_samples_in_window.read() / ol.gpio_dac_accumulate_num_accumulations.read()
    print('Dump rate:', dump_rate, ' Hz')

    dump_time = 1/dump_rate
    print('Dump time:', dump_time*1000, ' ms')

    #print(np.max(np.abs(accumulations_adc0/num_accumulations)),np.max(np.abs(accumulations_adc1/num_accumulations)))
#     time.sleep(1/60) # let the board look for keyboard input
#     time.sleep(1/2) # for a rough number of frames per second FPS